- [A. Préparation des données](#A)
	- [A.1. Importation des modules Python](#A.1)
	- [A.2. Chargement du jeu de données](#A.2)
	- [A.3. Nettoyage des données](#A.3)
		- [A.3.1. Traitement des valeurs manquantes](#A.3.1)
		- [A.3.2. Encodage des variables catégorielles](#A.3.2)
		- [A.3.3. Suppression des doublons](#A.3.3)
	- [A.4. Exploration des données](#A.4)
		- [A.4.1. Tests statistiques](#A.4.1)
		- [A.4.2. Visualisation des distributions](#A.4.2)
		- [A.4.3. Détection et correction des anomalies: Traitement des valeurs aberrantes](#A.4.3)
		- [A.4.4. Analyse et visualisation des corrélations](#A.4.4)
		- [A.4.5. Normalisation des données](#A.4.5)
		- [A.4.6. Analyse statistique descriptive](#A.4.6)
	- [A.5. Features Engineering](#A.5)
	- [A.6. Séparation des données](#A.6)
		- [A.6.1. Localisation de la variable cible Attrition](#A.6.1)
		- [A.6.2. Division en ensembles d'apprentissage et de l'ensemble learning](#B.4)

## <a id='A'>A. Préparation des données</a>

### <a id='A.1'>A.1. Importation des modules Python</a>

In [1]:
import numpy as np
from openpyxl import load_workbook
import pandas as pd
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')

C:\Users\DELL\AppData\Local\Temp\ipykernel_12144\1485602306.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### <a id='A.2'>A.2. Chargement du jeu de données</a>

In [2]:
# Load workbook
raw_data_filepath = "../../data/raw/Exam_ML_ISE-2.xlsx"
wb = load_workbook(filename=raw_data_filepath)
print(f'Nom des feuilles du classeur excel: {wb.sheetnames}')

Nom des feuilles du classeur excel: ['Présentation du Jeux de données', 'Description des Variables', 'JeuxDeDonnées']


In [3]:
# Access to a worksheet named 'sample'
ws = wb[wb.sheetnames[-1]]

# Get the first line in file as a header line
values = ws.values
columns = next(values)[0:]

# Create a DataFrame based on the second and subsequent lines of data
data = pd.DataFrame(values, columns=columns)

In [4]:
data.head()

,Age,Attrition,Voyage_affaires,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,Satisfaction_relationnelle,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1.0,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8.0,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2.0,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3.0,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2.0,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


### <a id='A.3'>A.3. Nettoyage des données</a>

#### <a id='A.3.1'>A.3.1. Traitement des valeurs manquantes</a>

In [ ]:
#### <a id='A.3.2'>A.3.2. Encodage des variables catégorielles</a>

In [ ]:
#### <a id='A.3.3'>A.3.3. Suppression des doublons</a>

In [ ]:
### <a id='A.4'>A.4. Exploration des données</a>

In [ ]:
#### <a id='A.4.1'>A.4.1. Tests statistiques</a>

In [ ]:
#### <a id='A.4.2'>A.4.2. Visualisation des distributions</a>

In [ ]:
#### <a id='A.4.3'>A.4.3. Détection et correction des anomalies: Traitement des valeurs aberrantes</a>

In [ ]:
#### <a id='A.4.4'>A.4.4. Analyse et visualisation des corrélations</a>

In [ ]:
#### <a id='A.4.5'>A.4.5. Normalisation des données</a>

In [ ]:
#### <a id='A.4.6'>A.4.6. Analyse statistique descriptive</a>

In [ ]:
### <a id='A.5'>A.5. Features Engineering</a>

In [ ]:
### <a id='A.6'>A.6. Séparation des données</a>

In [ ]:
#### <a id='A.6.1'>A.6.1. Localisation de la variable cible Attrition</a>

In [ ]:
#### <a id='A.6.2'>A.6.2. Division en ensembles d'apprentissage et de test</a>